In [1]:
# Necessary imports
%load_ext autoreload
%autoreload 2

import networkx as nx
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.backend as K

from TCGAMultiOmics.multiomics import MultiOmicsData
from moge.network.heterogeneous_network import HeterogeneousNetwork

from moge.visualization.plot_data import matrix_heatmap

Using TensorFlow backend.


In [6]:
import pickle

# READ
with open('moge/data/LMN_future_recall/TRAIN/Interactions_Affinity/LMN_mirtarbase_biogrid_starbase_lncrna2target_lncrinter.train.pickle', 'rb') as file:
    network = pickle.load(file)
    
with open('moge/data/LMN_future_recall/TEST/Interactions_Affinity/LMN_mirtarbase_biogrid_starbase_lncrna2target_lncrinter.test.pickle', 'rb') as file:
    network_val = pickle.load(file)

In [3]:
from moge.embedding.siamese_triplet_online_embedding import SiameseOnlineTripletGraphEmbedding, OnlineTripletLoss

siamese = SiameseOnlineTripletGraphEmbedding(batch_size=256, verbose=True)

In [7]:
siamese.learn_embedding(network=network, network_val=network_val, multi_gpu=False)

Epoch 1/10
500/500 [==============================] - 869s 2s/step - loss: 0.0163
Epoch 2/10
132/500 [======>.......................] - ETA: 10:55 - loss: 0.0200Stop training


Process ForkPoolWorker-33:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/opt/anaconda3/lib/python3.6/site-packages/keras/utils/data_utils.py", line 401, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/home/jonny/PycharmProjects/MultiOmicsGraphEmbedding/moge/network/triplet_generator.py", line 106, in __getitem__
    X, y = self.__data_generation(sampled_nodes)
  File "/home/jonny/PycharmProjects/MultiOmicsGraphEmbedding/moge/network/triplet_generator.py", line 113, in __data_generation
    sampled_directed_adj = self.sample_directed_negative_edges(self.network.get_adjacency_matrix(edge_types=["d"], node_list=sampled_nodes))
  

  File "/home/jonny/PycharmProjects/MultiOmicsGraphEmbedding/moge/network/heterogeneous_network.py", line 150, in <listcomp>
    edge_list = [(u, v, d) for u, v, d in self.G.edges(data=True) if d['type'] in edge_types]
  File "/opt/anaconda3/lib/python3.6/site-packages/networkx/classes/reportviews.py", line 667, in <genexpr>
    for nbr, dd in nbrs.items())
  File "/opt/anaconda3/lib/python3.6/site-packages/networkx/classes/reportviews.py", line 655, in <lambda>
    self._report = lambda n, nbr, dd: (n, nbr, dd)
KeyboardInterrupt


In [9]:
X, y = siamese.generator_train.__getitem__(1)
X["labels_directed"]

(matrix([[  0, 285],
         [  0, 288],
         [  0, 399],
         ...,
         [999, 310],
         [999, 509],
         [999, 792]], dtype=int32),
 array([1.e-16, 1.e-16, 1.e-16, ..., 1.e-16, 1.e-16, 1.e-16]),
 (1000, 1000))

In [76]:
a = tf.SparseTensor(*X["labels_directed"])

In [77]:
tf.sparse_tensor_to_dense( a, default_value = False )

<tf.Tensor: id=131, shape=(50, 50), dtype=float64, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])>

In [81]:
e = sparse_greater_than(a, c=0.5)
e

In [82]:
e.indices

<tf.Tensor: id=141, shape=(7, 2), dtype=int64, numpy=
array([[ 8, 16],
       [14, 39],
       [18, 47],
       [33,  8],
       [33, 16],
       [33, 41],
       [33, 48]])>

In [83]:
tf.sparse_tensor_to_dense( e, default_value = False )

<tf.Tensor: id=149, shape=(50, 50), dtype=bool, numpy=
array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])>

In [80]:
def sparse_greater_than(s, c=0.5):
    a = tf.sparse_retain(s, tf.greater(s.values, c))
    return tf.SparseTensor(a.indices, 
                           tf.ones_like(a.values, dtype=tf.bool), 
                           s.dense_shape)

def sparse_not_equal(a, b):
    neg_b = tf.SparseTensor( b.indices, -b.values, dense_shape = b.dense_shape )
    difference = tf.sparse_add( a, neg_b )
    return sparse_not_equal_to_constant( difference, 0.0 )

# # test data
# a = tf.SparseTensor( [ [ 0, 0 ], [ 1, 4 ], [ 2, 3 ] ], [ 5.0, 6, 7 ], 
#                     dense_shape = ( 5, 5 ) )
# b = tf.SparseTensor( [ [ 0, 0 ], [ 0, 2 ], [ 2, 3 ] ], [ 5.0, 6, 2 ], 
#                     dense_shape = ( 5, 5 ) )
# e = sparse_not_equal( a, b )
# f = tf.sparse_tensor_to_dense( e, default_value = False )
# f

In [ ]:
tf.random.learned_unigram_candidate_sampler()

In [ ]:
tf.nn.learned_unigram_candidate_sampler(true_classes=,
    num_true=,
    num_sampled=1,
    unique=False,
    range_max=)